# RNN language model
Loosely based on [Zaremba et al. 2014](https://arxiv.org/abs/1409.2329), this example trains a word based RNN language model on Mikolov's PTB data with 10K vocab. It uses the `batchSizes` feature of `rnnforw` to process batches with different sized sentences. The `mb` minibatching function sorts sentences in a corpus by length and tries to group similarly sized sentences together. For an example that uses fixed length batches and goes across sentence boundaries see the [charlm](https://github.com/denizyuret/Knet.jl/blob/master/tutorial/08.charlm.ipynb) notebook. **TODO:** convert to the new RNN interface.

In [1]:
EPOCHS=10
RNNTYPE=:lstm
BATCHSIZE=64
EMBEDSIZE=128
HIDDENSIZE=256
VOCABSIZE=10000
NUMLAYERS=1
DROPOUT=0.5
LR=0.001
BETA_1=0.9
BETA_2=0.999
EPS=1e-08;

In [2]:
# Load data
using Knet
include(Knet.dir("data","mikolovptb.jl"))
(trn,val,tst,vocab) = mikolovptb()
@assert VOCABSIZE == length(vocab)+1 # +1 for the EOS token
for x in (trn,val,tst,vocab); println(summary(x)); end

42068-element Array{Array{UInt16,1},1}
3370-element Array{Array{UInt16,1},1}
3761-element Array{Array{UInt16,1},1}
9999-element Array{String,1}


In [3]:
# Print a sample
println(tst[1])
println(vocab[tst[1]])

UInt16[0x008e, 0x004e, 0x0036, 0x00fb, 0x0938, 0x0195]
["no", "it", "was", "n't", "black", "monday"]


In [4]:
@doc mikolovptb

```
mikolovptb()
```

Read [PTB](https://catalog.ldc.upenn.edu/ldc99t42) text from Mikolov's [RNNLM](http://www.fit.vutbr.cz/~imikolov/rnnlm) toolkit which has been lowercased and reduced to a 10K vocabulary size.  Return a tuple (trn,dev,tst,vocab) where

```
trn::Vector{Vector{UInt16}}: 42068 sentences, 887521 words
dev::Vector{Vector{UInt16}}: 3370 sentences, 70390 words
tst::Vector{Vector{UInt16}}: 3761 sentences, 78669 words
vocab::Vector{String}: 9999 unique words
```


In [5]:
# Minibatch data into (x,y,b) triples. This is the most complicated part of the code:
# for language models x and y contain the same words shifted, x has an EOS in the beginning, y has an EOS at the end
# x,y = [ s11,s21,s31,...,s12,s22,...] i.e. all the first words followed by all the second words etc.
# b = [b1,b2,...,bT] i.e. how many sentences have first words, how many have second words etc.
# length(x)==length(y)==sum(b) and length(b)=length(s1)+1 (+1 because of EOS)
# sentences in batch should be sorted from longest to shortest, i.e. s1 is the longest sentence
function mb(sentences,batchsize)
    sentences = sort(sentences,by=length,rev=true)
    data = []; eos = VOCABSIZE
    for i = 1:batchsize:length(sentences)
        j = min(i+batchsize-1,length(sentences))
        sij = view(sentences,i:j)
        T = 1+length(sij[1])
        x = UInt16[]; y = UInt16[]; b = UInt16[]
        for t=1:T
            bt = 0
            for s in sij
                if t == 1
                    push!(x,eos)
                    push!(y,s[1])
                elseif t <= length(s)
                    push!(x,s[t-1])
                    push!(y,s[t])
                elseif t == 1+length(s)
                    push!(x,s[t-1])
                    push!(y,eos)
                else
                    break
                end
                bt += 1
            end
            push!(b,bt)
        end
        push!(data,(x,y,b))
    end
    return data
end

mbtrn = mb(trn,BATCHSIZE)
mbval = mb(val,BATCHSIZE)
mbtst = mb(tst,BATCHSIZE)
map(length,(mbtrn,mbval,mbtst))

(658, 53, 59)

In [10]:
# Define model
function initmodel()
    w(d...)=KnetArray(xavier(Float32,d...))
    b(d...)=KnetArray(zeros(Float32,d...))
    r,wr = rnninit(EMBEDSIZE,HIDDENSIZE,rnnType=RNNTYPE,numLayers=NUMLAYERS,dropout=DROPOUT,atype=KnetArray{Float32})
    wx = w(EMBEDSIZE,VOCABSIZE)
    wy = w(VOCABSIZE,HIDDENSIZE)
    by = b(VOCABSIZE,1)
    return r,wr,wx,wy,by
end;

In [14]:
# Define loss and its gradient
function predict(ws,xs,bs;pdrop=0)
    r,wr,wx,wy,by = ws
    r = value(r)
    x = wx[:,xs] # xs=(ΣBt) x=(X,ΣBt)
    x = dropout(x,pdrop)
    (y,_) = rnnforw(r,wr,x,batchSizes=bs) # y=(H,ΣBt)
    y = dropout(y,pdrop)
    return wy * y .+ by  # return=(V,ΣBt)
end

loss(w,x,y,b;o...) = nll(predict(w,x,b;o...), y)

lossgradient = gradloss(loss);

In [15]:
# Train and test loops
function train(model,data,optim)
    Σ,N=0,0
    for (x,y,b) in data
        grads,loss1 = lossgradient(model,x,y,b;pdrop=DROPOUT)
        update!(model, grads, optim)
        n = length(y)
        Σ,N = Σ+n*loss1, N+n
    end
    return Σ/N
end

function test(model,data)
    Σ,N=0,0
    for (x,y,b) in data
        loss1 = loss(model,x,y,b)
        n = length(y)
        Σ,N = Σ+n*loss1, N+n
    end
    return Σ/N
end;

In [16]:
model = optim = nothing; 
Knet.gc() # free gpu memory
if !isfile("rnnlm.jld2")
    # Initialize and train model
    model = initmodel()
    optim = optimizers(model,Adam,lr=LR,beta1=BETA_1,beta2=BETA_2,eps=EPS)
    for epoch=1:EPOCHS
        @time global j1 = train(model,mbtrn,optim)  # ~100 seconds
        @time global j2 = test(model,mbval)         # ~4 seconds
        @time global j3 = test(model,mbtst)         # ~4 seconds
        println((epoch,exp(j1),exp(j2),exp(j3))); flush(stdout)  # prints perplexity = exp(negative_log_likelihood)
    end
    Knet.save("rnnlm.jld2","model",model)
else
    model = Knet.load("rnnlm.jld2","model")
    @time global j1 = test(model,mbtrn)
    @time global j2 = test(model,mbval)
    @time global j3 = test(model,mbtst)
    println((EPOCHS,exp(j1),exp(j2),exp(j3))); flush(stdout)  # prints perplexity = exp(negative_log_likelihood)
end
summary(model)

 27.326092 seconds (21.46 M allocations: 1.116 GiB, 3.64% gc time)
  0.900037 seconds (1.05 M allocations: 57.092 MiB, 8.54% gc time)
  0.317074 seconds (36.53 k allocations: 3.560 MiB, 6.61% gc time)
(1, 661.0763f0, 8.268473f6, 7.955551f6)
 13.010777 seconds (1.99 M allocations: 121.218 MiB, 3.83% gc time)
  0.323326 seconds (32.78 k allocations: 3.189 MiB, 7.95% gc time)
  0.350242 seconds (36.61 k allocations: 3.571 MiB, 8.48% gc time)
(2, 561.70776f0, 60723.773f0, 57029.04f0)
 13.460838 seconds (1.99 M allocations: 121.222 MiB, 3.70% gc time)
  0.275338 seconds (32.83 k allocations: 3.182 MiB, 8.25% gc time)
  0.296452 seconds (36.66 k allocations: 3.576 MiB, 8.44% gc time)
(3, 402.2791f0, 12841.272f0, 12044.729f0)
 13.777386 seconds (1.99 M allocations: 121.357 MiB, 4.60% gc time)
  0.320279 seconds (32.88 k allocations: 3.207 MiB, 11.58% gc time)
  0.340023 seconds (36.61 k allocations: 3.540 MiB, 9.03% gc time)
(4, 334.88797f0, 6216.086f0, 5853.3613f0)
 12.735467 seconds (1.99 M

"(LSTM(input=128,hidden=256,dropout=0.5), K32(1,1,395264)[-0.07042014⋯], K32(128,10000)[0.006436585⋯], K32(10000,256)[-0.061549857⋯], K32(10000,1)[-0.08249636⋯])"